In [1]:
import numpy as np
import pandas as pd
from scipy.stats import mannwhitneyu

In [2]:
def bb_to_percentage(value):
    bb_dict = {
        'r': 0,
        '+': 0.3,
        '1': 3,
        '2a': 9,
        '2b': 19,
        '3': 32,
        '4': 57,
        '5': 90
    }
    if isinstance(value, str) and value in bb_dict:
        return bb_dict[value]
    else:
        return np.nan

In [3]:
def apply_bb_to_column(df, col_index):
    col = df.iloc[:, col_index]
    transformed_col = []
    for value in col:
        transformed_value = bb_to_percentage(value)
        transformed_col.append(transformed_value)
    result_array = np.array(transformed_col)
    return result_array

In [4]:
def get_landolt_values(species, landolt_data):
    landolt_values = landolt_data[landolt_data['Espece'].isin(species)]
    return landolt_values[['Espece', 'T', 'L', 'K', 'R', 'N', 'F']]

def calculate_weighted_landolt(species, coverages, landolt_data):
    landolt_values = get_landolt_values(species, landolt_data)
    
    # Associer les recouvrements aux espèces correspondantes
    species_coverage = pd.DataFrame({'Espece': species, 'Coverage': coverages})
    combined_data = pd.merge(species_coverage, landolt_values, on='Espece', how='left')

    # Convertir les colonnes en types numériques
    for col in ['T', 'L', 'K', 'R', 'N', 'F']:
        combined_data[col] = pd.to_numeric(combined_data[col], errors='coerce')
    
    # Calculer les moyennes pondérées
    weighted_landolt = {}
    weighted_error = {}
    total_coverage = combined_data['Coverage'].sum()
    if total_coverage == 0:
        return {col: (np.nan, np.nan) for col in ['T', 'L', 'K', 'R', 'N', 'F']}
    
    for col in ['T', 'L', 'K', 'R', 'N', 'F']:
        combined_data[col + '_weighted'] = combined_data[col] * combined_data['Coverage']
        mean = combined_data[col + '_weighted'].sum() / total_coverage
        weighted_landolt[col] = mean
        
        # Calcul de l'erreur standard pondérée
        variance = combined_data.apply(lambda row: (row[col] - mean) ** 2 * row['Coverage'], axis=1).sum() / total_coverage
        standard_error = np.sqrt(variance) / np.sqrt(total_coverage)
        weighted_error[col] = standard_error
    
    # Retourner les moyennes et les erreurs standard pondérées
    return {col: (weighted_landolt[col], weighted_error[col]) for col in ['T', 'L', 'K', 'R', 'N', 'F']}


In [5]:
def main():
    file_path = 'Végétation excursions.xlsx'  # Remplacez par le chemin de votre fichier
    flora_data_path = 'FloraIndicativa_clean_R.xlsx'
    flora_data_sheet = 'vasc'
    
    # Lire le fichier principal
    df = pd.read_excel(file_path)
    
    # Appliquer la transformation de recouvrement sur la colonne 3
    coverage = apply_bb_to_column(df, 3)
    print("Recouvrement transformé:", coverage)
    
    # Extraire les espèces de la colonne 2
    species = df.iloc[:, 2].values
    print("Espèces:", species)
    
    # Extraire les sites de la colonne 0
    sites = df.iloc[:, 0].values
    print("Sites:", sites)
    
    # Lire les données de FloraIndicativa
    landolt_data = pd.read_excel(flora_data_path, sheet_name=flora_data_sheet)
    
    # Calculer les indices de Landolt pondérés pour chaque site
    site_landolt_indices = {}
    unique_sites = np.unique(sites)
    
    for site in unique_sites:
        site_mask = sites == site
        site_species = species[site_mask]
        site_coverage = coverage[site_mask]
        site_landolt_indices[site] = calculate_weighted_landolt(site_species, site_coverage, landolt_data)
    
if __name__ == "__main__":
    main()

Recouvrement transformé: [ 9.   9.   0.3  9.  32.  32.   9.   3.   0.   0.   0.   0.   0.3 32.
  3.   0.   0.3  nan  9.   0.3  0.3  0.3 32.   3.   0.  57.  19.   0.
  0.3  0.3  0.   nan  nan  0.3 57.   0.3  3.   0.3  0.3  0.3  0.3  3.
  0.3  0.3  0.3  9.   nan  0.3  0.3  0.3  0.3  0.3  0.   0.3  0.  57.
  0.3  0.3  0.   0.   0.3  0.   0.3  3.  19.   3.   0.3  9.   9.  19.
 19.   0.3  0.3  0.3  0.3  9.   9.   3.   3.   3.   3.   3.   0.3  0.3
  0.3  0.3  0.3 57.   3.   3.   3.   0.3  0.3  0.3  0.3  0.3  0.3  0.3
  0.   0. ]
Espèces: ['Rubus fruticosus aggr.' 'Salix aurita' 'Salix purpurea'
 'Equisetum palustre' 'Rubus fruticosus aggr.' 'Phragmites australis'
 'Cirsium arvense' 'Lysimachia vulgaris' 'Filipendula ulmaria'
 'Thalictrum flavum' 'Symphytum officinale' 'Cornus sanguinea'
 'Rorippa palustris' 'Solidago gigantea' 'Carex hirta' 'Quercus sp.'
 'Salix purpurea' '(Alnus glutinosa)' 'Lysimachia vulgaris'
 'Symphytum officinale' 'Galium palustre' 'Hydrocotyle vulgaris'
 'Cladium mari

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
